In [18]:
from keras.applications.vgg16 import preprocess_input 

# models 
from keras.applications.vgg16 import VGG16 
from keras.models import Model

# clustering and dimension reduction
from sklearn.cluster import KMeans, MeanShift, SpectralClustering
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

import os
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import pickle
import random

from PIL import Image

from skimage.io import imread, imsave
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm

pd.set_option('display.max_rows', None)

In [2]:
pip install scikit-image

Note: you may need to restart the kernel to use updated packages.


In [3]:
model = VGG16()
model = Model(inputs = model.inputs, outputs = model.layers[-2].output)

2022-06-09 12:00:20.301300: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-09 12:00:53.158360: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7434 MB memory:  -> device: 0, name: Tesla M60, pci bus id: 0001:00:00.0, compute capability: 5.2


In [4]:
def extract_features(img_name, image_folder, model):
    white = (255,255,255)
    black = (0,0,0)
    # load the image as a 224x224 array
    img_path = f"{image_folder}/{img_name}"
    #img = load_img(img_path, target_size=(224,224))
    img = Image.open(img_path)
    img = img.resize((224,224))
    img = np.asarray(img)
    imRGB = np.repeat(img[:, :, np.newaxis], 3, axis=2)
    
    # reshape the data for the model reshape(num_of_samples, dim 1, dim 2, channels)
    reshaped_img = imRGB.reshape(1,224,224, 3) 
    # prepare image for model
    imgx = preprocess_input(reshaped_img)
    # get the feature vector
    features = model.predict(imgx, use_multiprocessing=True)
    return features

In [5]:
data_X = {}
data_Y = {}
dirs = ['i', 'ii', 'iii', 'iv', 'ix', 'v', 'vi', 'vii', 'viii', 'x']
main_dir = 'all_data'
# lop through each image in the dataset
for d in dirs:
    for img_name in os.listdir(f"{main_dir}/{d}"):
        if img_name != ".DS_Store":
            feat = extract_features(img_name,f"{main_dir}/{d}",model)
            data_X[img_name] = feat
            data_Y[img_name] = d

2022-06-09 12:02:07.732329: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8400


1/1 [==============================] - 0s 19ms/step


1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 19ms/step


1/1 [==============================] - 0s 20ms/step


In [6]:
filenames = np.array(list(data_X.keys()))
feat = np.array(list(data_X.values()))
feat.shape

(2077, 1, 4096)

In [7]:
feat = feat.reshape(-1,4096)
feat.shape

(2077, 4096)

In [9]:
pca = PCA(random_state=22)
pca.fit(feat)
x = pca.transform(feat)

### data v2

##### use the VGG16 model to relabel the images + adding extra images to the train (cropping)

In [23]:
classes = ['i', 'ii', 'iii', 'iv', 'ix', 'v', 'vi', 'vii', 'viii', 'x']
train_frac = [0.99, 0.95, 0.9, 0.85, 0.8, 0.75]
mapping = pd.read_csv('mapping_images_name.csv')

for k in [2]:
    neigh = KNeighborsClassifier(n_neighbors=k)
    y = list(data_Y.values())
    neigh.fit(x, y)
    new_y = neigh.predict(x)
    for i in range(4):
        new_folder_name = f"data_v2/KNN_{k}_iter_{i}"
        os.mkdir(new_folder_name)
        os.mkdir(f"{new_folder_name}/train")
        os.mkdir(f"{new_folder_name}/val")

        # create list with all the relvant images.
        image_names = []
        for i, img_name in enumerate(filenames):
            if y[i] == new_y[i]:
                image_names.append(img_name)
        frac_use = len(image_names)/len(filenames)
        print(f"Images to use in {k}-NN: {frac_use}")
        mapping_r = mapping[mapping['new_name'].isin(image_names)]
        for c in classes:
            images_c = mapping_r[mapping_r['folder'] == c]['new_name'].to_list()
            random.shuffle(images_c)
            x_test = images_c[:50]
            x_train = images_c[50:]

            old_path = f"all_data/{c}"
            new_path_val = f"{new_folder_name}/val/{c}"
            new_path_train = f"{new_folder_name}/train/{c}"

            os.mkdir(new_path_val)
            os.mkdir(new_path_train)

            for img_name in x_test: 
                img = imread(f"{old_path}/{img_name}")
                imsave(f"{new_path_val}/{img_name}", img)
                
            for img_name in x_train: 
                img = imread(f"{old_path}/{img_name}")
                imsave(f"{new_path_train}/{img_name}", img)
            
            counter_train = len(x_train)
            for img_name in x_train:
                img_arr = imread(f"{new_path_train}/{img_name}")
                height, width = img_arr.shape[0], img_arr.shape[1]
                border_w = width // 10
                border_h = height // 10
                img_wo_border = img_arr[border_h:height-border_h, border_w:width-border_w]
                height, width = img_wo_border.shape[0], img_wo_border.shape[1]
                for i in range(0, 2):
                    for j in range(0, 2):
                        cropped_img = img_wo_border[j:j+(height-2), i:i+(width-2)]
                        imsave(f"{new_path_train}/{i}_{j}_{img_name}", cropped_img)
                        counter_train += 1
                        if counter_train == 950:
                            break
                    if counter_train == 950:
                        break
                if counter_train == 950:
                    break
                            

Images to use in 2-NN: 0.8257101588830044


/tmp/ipykernel_21824/2396401177.py:39: UserWarning: data_v2/KNN_2_iter_0/val/i/i_63.png is a low contrast image
  imsave(f"{new_path_val}/{img_name}", img)
/tmp/ipykernel_21824/2396401177.py:43: UserWarning: data_v2/KNN_2_iter_0/train/i/i_245.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_21824/2396401177.py:43: UserWarning: data_v2/KNN_2_iter_0/train/i/i_254.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_21824/2396401177.py:43: UserWarning: data_v2/KNN_2_iter_0/train/i/i_110.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_21824/2396401177.py:43: UserWarning: data_v2/KNN_2_iter_0/train/i/i_180.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_21824/2396401177.py:43: UserWarning: data_v2/KNN_2_iter_0/train/i/i_174.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_21824/2396401177.py:43: 

Images to use in 2-NN: 0.8257101588830044


/tmp/ipykernel_21824/2396401177.py:39: UserWarning: data_v2/KNN_2_iter_1/val/i/i_174.png is a low contrast image
  imsave(f"{new_path_val}/{img_name}", img)
/tmp/ipykernel_21824/2396401177.py:39: UserWarning: data_v2/KNN_2_iter_1/val/i/i_79.png is a low contrast image
  imsave(f"{new_path_val}/{img_name}", img)
/tmp/ipykernel_21824/2396401177.py:39: UserWarning: data_v2/KNN_2_iter_1/val/i/i_110.png is a low contrast image
  imsave(f"{new_path_val}/{img_name}", img)
/tmp/ipykernel_21824/2396401177.py:43: UserWarning: data_v2/KNN_2_iter_1/train/i/i_127.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_21824/2396401177.py:43: UserWarning: data_v2/KNN_2_iter_1/train/i/i_250.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_21824/2396401177.py:43: UserWarning: data_v2/KNN_2_iter_1/train/i/i_254.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_21824/2396401177.py:43: UserWarn

Images to use in 2-NN: 0.8257101588830044


/tmp/ipykernel_21824/2396401177.py:39: UserWarning: data_v2/KNN_2_iter_2/val/i/i_110.png is a low contrast image
  imsave(f"{new_path_val}/{img_name}", img)
/tmp/ipykernel_21824/2396401177.py:39: UserWarning: data_v2/KNN_2_iter_2/val/i/i_174.png is a low contrast image
  imsave(f"{new_path_val}/{img_name}", img)
/tmp/ipykernel_21824/2396401177.py:39: UserWarning: data_v2/KNN_2_iter_2/val/i/i_254.png is a low contrast image
  imsave(f"{new_path_val}/{img_name}", img)
/tmp/ipykernel_21824/2396401177.py:43: UserWarning: data_v2/KNN_2_iter_2/train/i/i_79.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_21824/2396401177.py:43: UserWarning: data_v2/KNN_2_iter_2/train/i/i_63.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_21824/2396401177.py:43: UserWarning: data_v2/KNN_2_iter_2/train/i/i_180.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_21824/2396401177.py:43: UserWarni

Images to use in 2-NN: 0.8257101588830044


/tmp/ipykernel_21824/2396401177.py:39: UserWarning: data_v2/KNN_2_iter_3/val/i/i_254.png is a low contrast image
  imsave(f"{new_path_val}/{img_name}", img)
/tmp/ipykernel_21824/2396401177.py:43: UserWarning: data_v2/KNN_2_iter_3/train/i/i_210.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_21824/2396401177.py:43: UserWarning: data_v2/KNN_2_iter_3/train/i/i_63.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_21824/2396401177.py:43: UserWarning: data_v2/KNN_2_iter_3/train/i/i_174.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_21824/2396401177.py:43: UserWarning: data_v2/KNN_2_iter_3/train/i/i_57.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_21824/2396401177.py:43: UserWarning: data_v2/KNN_2_iter_3/train/i/i_180.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_21824/2396401177.py:43: U